# LangChain/LangGraph로 구현하는 에이전트 디자인 패턴

## 환경 설정

In [1]:
!git clone https://github.com/ychoi-kr/llm-agent.git

Cloning into 'llm-agent'...
remote: Enumerating objects: 635, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 635 (delta 154), reused 130 (delta 130), pack-reused 459 (from 1)
Receiving objects: 100% (635/635), 1.91 MiB | 8.83 MiB/s, done.
Resolving deltas: 100% (344/344), done.


In [2]:
%cd llm-agent
%cd chapter12

/Users/iyeongbeom/YB/Repository/study/llm-agent/chapter12/llm-agent
/Users/iyeongbeom/YB/Repository/study/llm-agent/chapter12/llm-agent/chapter12


In [3]:
!pip install langchain-core==0.3.0 langchain-community==0.3.0 \
  langgraph==0.2.22 langchain-openai==0.2.0 langchain-anthropic==0.2.0 \
    python-dotenv==1.0.1 numpy==1.26.4 faiss-cpu==1.8.0.post1 \
      pydantic-settings==2.5.2 retry==0.9.2 decorator==4.4.2 pydantic==2.9.2

  Using cached langchain_core-0.3.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
Using cached langchain_core-0.3.0-py3-none-any.whl (405 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 7.9 MB/s  0:00:00 eta 0:00:01m
Using cached defusedxml-0.7.1-py2.py3-n

In [ ]:
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
# os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
# os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [1]:
import os
# from google.colab import userdata
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

## 1. 패시브 골 크리에이터

사용자의 입력에서 구체적인 목표를 추출하는 패턴. 여기서는 간단하게 프롬프트만을 사용해 사용자 입력을 '목표'의 내용이 표현된 데이터 모델의 객체로 변환하는 형태로 구현

In [ ]:
!python -m passive_goal_creator.main --task "AI agent 만들기 실습"

사용자가 'simplePrompt'라는 입력을 제공했습니다. 이 입력은 구체적인 목표를 생성하기 위한 충분한 정보가 아닙니다. 따라서, 사용자가 원하는 목표의 방향성을 파악하기 위해 추가적인 정보를 요청하거나, 'simplePrompt'라는 키워드와 관련된 일반적인 목표를 설정할 수 있습니다. 예를 들어, 'simplePrompt'가 특정 주제나 프로젝트와 관련이 있다면, 그에 맞는 목표를 설정할 수 있습니다. 현재로서는 'simplePrompt'라는 키워드에 대한 조사를 수행하여 관련 정보를 수집하고, 이를 바탕으로 사용자가 원하는 목표를 구체화하는 것이 필요합니다.


**모호한 요청이 구체적인 목표로 개선**

## 2. 프롬프트/응답 최적화

생성된 목표나 사용자 요청을 보다 효과적인 프롬프트로 변환하고, LLM으로부터 더 높은 품질의 답변을 얻기 위한 패턴
- 입력된 목표를 바탕으로 효과적인 프롬프트를 생성하는 프로그램(프롬프트 최적화)
- 입력된 목표를 바탕으로 최적의 응답 내용을 정의하는 프로그램(응답 최적화)

In [9]:
!python -m prompt_optimizer.main --task "AI agent 만들기 실습"

AI 에이전트를 만들기 위한 실습 목표를 설정합니다. 이 목표는 AI 에이전트를 개발하는 데 필요한 기본적인 지식과 기술을 습득하는 것을 목표로 합니다. 이를 위해 다음과 같은 구체적인 단계를 포함합니다: 1) AI 에이전트의 기본 개념과 작동 원리를 이해하기 위한 이론적 학습을 진행합니다. 2) Python과 같은 프로그래밍 언어를 사용하여 간단한 AI 에이전트를 구현하는 실습을 수행합니다. 3) AI 에이전트의 성능을 평가하고 개선하기 위한 방법을 학습합니다. 4) 인터넷을 통해 관련 자료를 조사하고, 최신 AI 기술 동향을 파악합니다. 5) 실습 결과를 바탕으로 보고서를 작성하여 학습 내용을 정리합니다.(측정 기준: 1) AI 에이전트의 기본 개념과 작동 원리에 대한 이론적 학습을 완료했는지 확인하기 위해 퀴즈를 통해 평가합니다. 2) Python을 사용하여 간단한 AI 에이전트를 성공적으로 구현했는지 코드 리뷰를 통해 확인합니다. 3) AI 에이전트의 성능 평가 및 개선 방법을 이해했는지 실습 결과를 바탕으로 한 보고서를 통해 평가합니다. 4) 최신 AI 기술 동향을 조사하고 파악했는지 조사 보고서를 통해 확인합니다. 5) 최종 보고서를 통해 전체 학습 내용을 정리하고, 피드백을 통해 학습 목표 달성 여부를 평가합니다.)


In [10]:
!python -m response_optimizer.main --task "AI agent 만들기 실습"

목표 분석:
이 목표는 AI 에이전트의 기본 개념과 작동 원리를 이해하고, 이를 바탕으로 간단한 AI 에이전트를 설계 및 구현하는 과정을 포함합니다. 주요 요소는 학술 논문과 강의를 통한 이론적 이해, Python, TensorFlow, PyTorch를 활용한 실습, 챗봇 기능을 가진 AI 에이전트의 설계 및 구현, 그리고 실습 결과를 바탕으로 한 보고서 작성입니다. 의도는 이론과 실습을 통해 AI 에이전트에 대한 포괄적인 이해를 얻고, 이를 바탕으로 실제 구현 능력을 배양하는 것입니다.

응답 사양:
응답은 명확하고 체계적으로 구성되어야 하며, 각 단계별로 필요한 정보와 지침을 제공해야 합니다. 톤은 교육적이고 격려적이어야 하며, 사용자가 각 단계를 이해하고 수행할 수 있도록 돕는 데 초점을 맞춰야 합니다. 내용은 이론적 배경, 실습 지침, 구현 방법, 테스트 및 보고서 작성 방법을 포함해야 합니다.

AI 에이전트에 대한 지침:
1. AI 에이전트의 기본 개념과 작동 원리를 설명하는 10개의 학술 논문과 5개의 온라인 강의를 요약합니다. 각 요약은 핵심 개념과 주요 내용을 포함해야 합니다.
2. Python, TensorFlow, PyTorch의 기본 사용법을 설명하고, 각 도구로 'Hello World' 수준의 간단한 프로그램을 작성하는 방법을 안내합니다.
3. 챗봇 기능을 가진 간단한 AI 에이전트를 설계하고 구현하는 방법을 단계별로 설명합니다. 주어진 5개의 시나리오에 대해 테스트하는 방법도 포함합니다.
4. 실습 결과를 바탕으로 10페이지 분량의 보고서를 작성하는 방법을 안내합니다. 보고서에는 학습 내용, 각 단계에서의 문제점과 해결 방법이 포함되어야 합니다.

응답 예시:
1. "AI 에이전트의 기본 개념을 이해하기 위해, 다음과 같은 논문과 강의를 참고하세요: [논문 및 강의 목록]. 각 자료의 요약은 다음과 같습니다..."
2. "Python, TensorFlow, PyTorch로 'Hello World' 프로그램을 작성하려면, 다음 단계를 따

## 3. 싱글 패스 플랜 제너레이터

설정된 목표를 달성하기 위한 일련의 구체적인 단계를 생성하는 패턴
구현된 AI 에이전트는 다음 작동을 확인
- 패시브 골 크리에이터와 프롬프트 최적화를 통한 목표 설정
- 설정된 목표에 대해 원샷 모델 쿼리와 싱글 패스 플랜 제너레이터를 사용해 목표를 개별 태스크로 분해
- 태스크 분해의 결과로 만들어진 태스크 리스트에 기반해 각 태스크를 실행
- 응답 최적화를 통해 정의된 응답 사양에 따른 보고서 출력

In [3]:
!python -m single_path_plan_generation.main --task "AI agent 만들기 실습"

🎬 Single Path Plan Generation 시작

🎯 [단계 1] 목표 설정 시작
  사용자 입력: AI agent 만들기 실습
  → 기본 목표 생성 중...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  ✓ 기본 목표: AI agent 만들기 실습을 위한 구체적이고 실행 가능한 목표는 다음과 같습니다. 1. AI 에이전트의 기본 개념과 작동 원리를 이해하기 위해 관련 자료를 조사합니다. 2. AI...
  → 목표 최적화 중 (SMART 원칙)...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  ✓ 최적화된 목표: 1. AI 에이전트의 기본 개념과 작동 원리를 이해하기 위해 3일 동안 관련 자료를 조사하고, 조사한 내용을 요약하여 1페이지 분량의 보고서를 작성한다. 2. AI 에이전트를 개발...
  ✓ 측정 기준: 1. 조사 보고서, 프로그래밍 언어 및 도구 선택 보고서, 설계 및 구현 계획서, 테스트 및 개선점 보고서, 최종 결과 보고서의 제출 여부를 확인한다. 2. 코딩 실습의 진행 상황...
  → 응답 형식 정의 중...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  ✓ 응답 형식 정의 완료
✅ [단계 1] 목표 설정 완료

📋 [단계 2] 목표 분해 시작
  목표: 1. AI 에이전트의 기본 개념과 작동 원리를 이해하기 위해 3일 동안 관련 자료를 조사하고, 조사한 내용을 요약하여 1페이지 분량의 보고서를 작성한다. 2. AI 에이전트를 개발...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
✅ 목표 분해 완료: 총 6개의 태스크 생성
  태스

## 4. 멀티 패스 플랜 제너레이터

태스크 분해 시 여러 선택지를 동시에 생성하고, 실행 시 컨텍스트에 따라 실행 에이전트 자신이 매번 적절한 선택을 하게 하는 패턴. 싱글 패스 플랜 제너레이터와의 차이점은, 설정된 목표에 대해 멀티 플랜 제너레이터를 통한 태스크 분해를 수행한다는 점.

In [3]:
!python -m multi_path_plan_generation.main --task "AI agent 만들기 실습"

프로그램 시작
모델: gpt-4o, 온도: 0.0
[MultiPathPlanGeneration] Multi-Path Plan Generation 시작
[MultiPathPlanGeneration] 입력 쿼리: AI agent 만들기 실습
[MultiPathPlanGeneration] 1단계: 목표 설정 시작
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[MultiPathPlanGeneration] 목표 생성 완료: AI agent 만들기 실습을 위한 구체적이고 실행 가능한 목표는 다음과 같습니다. 1. AI 에이전트의 기본 개념과 작동 방식을 이해하기 위해 관련 자료를 조사합니다. 2. AI 에이전트를 개발하기 위한 프로그래밍 언어와 도구를 선택합니다. 3. 간단한 AI 에이전트를 설계하고 구현하기 위한 계획을 세웁니다. 4. AI 에이전트의 성능을 테스트하고 개선할 수 있는 방법을 연구합니다. 5. 최종적으로 AI 에이전트를 완성하고, 결과를 보고서로 작성합니다.
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[MultiPathPlanGeneration] 목표 최적화 완료: 1. AI 에이전트의 기본 개념과 작동 방식을 이해하기 위해 10개의 관련 논문과 5개의 온라인 강의를 조사하고 요약한다. 2. AI 에이전트를 개발하기 위한 프로그래밍 언어와 도구를 Python과 TensorFlow로 선택하고, 각각의 기본 사용법을 학습한다. 3. 간단한 AI 에이전트를 설계하고 구현하기 위한 계획을 작성하고, 이를 기반으로 3주 내에 프로토타입을 개발한다. 4. AI 에이전트의 성능을 테스트하기 위해 5개의 테스트 케이스를 작성하고, 성능 개선을 위한 3가지 방법을 연구하여 적용한다. 5. 최종적으로 AI 에이전트를 완성하고, 결과를 10페이지 분량의 보고서로 작성하여 제출한

## 5. 셀프 리플렉션

태스크의 실행 결과를 에이전트 자신이 되돌아보고 그 실행 내용을 자체적으로 개선하는 패턴. 이번 구현에서는 돌아본 내용을 영속화 해 다음 실행 시에도 과거의 리플렉션 내용을 활용할 수 있도록 구현

In [4]:
!python -m self_reflection.main --task "AI agent 만들기 실습"

🎬 Self-Reflection Agent 시작
사용자 쿼리: AI agent 만들기 실습

🎯 [1단계: 목표 설정] 시작
🎯 [목표 설정] 과거 회고를 고려한 목표 생성 시작
  관련 과거 회고 0개 발견
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  기본 목표 생성 완료: AI 에이전트를 만들기 위한 실습을 진행하기 위해, 다음과 같은 구체적인 목표를 설정합니다: 1) AI 에이전트의 기본 개념과 작동 원리를 이해하기 위해 관련 자료를 조사합니다. ...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  목표 최적화 완료
📝 [응답 최적화] 과거 회고를 고려한 응답 형식 정의 시작
  관련 과거 회고 0개 발견
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  응답 형식 정의 완료
✅ [1단계: 목표 설정] 완료

📋 [2단계: 목표 분해] 시작
📋 [목표 분해] 과거 회고를 고려한 태스크 분해 시작
  관련 과거 회고 0개 발견
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  태스크 분해 완료: 총 5개의 태스크 생성
    태스크 1: 인터넷을 통해 AI 에이전트의 기본 개념과 작동 원리에 대한 10개의 학술 논문과 5개의 온라인 강의를 조사하고, 각 자료의 핵심 내용을 요약한...
    태스크 2: Python과 TensorFlow를 학습하기 위한 온라인 강의나 튜토리얼을 조사하고, 4주 내에 학습 계획을 수립하여 매일 학습을 진행한다....
    태스크 3: 간단한 AI 에이전트를 설계하고 구현하기 위한 단계별 계획을 수립하기 위해 관련 자료를 조사하고, 2주

## 6. 크로스 리플렉션

In [11]:
!python -m cross_reflection.main --task "AI agent 만들기 실습"

🔄 Cross-Reflection Agent 초기화
✅ OpenAI LLM 초기화 완료 (모델: gpt-4o)
✅ Anthropic LLM 초기화 완료 (모델: claude-sonnet-4-20250514)
📝 Cross-Reflection 설정: OpenAI가 실행, Anthropic이 성찰 수행

🎬 Self-Reflection Agent 시작
사용자 쿼리: AI agent 만들기 실습

🎯 [1단계: 목표 설정] 시작
🎯 [목표 설정] 과거 회고를 고려한 목표 생성 시작
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
  관련 과거 회고 2개 발견
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  기본 목표 생성 완료: 1. AI 에이전트의 정의와 기본 개념을 이해하기 위해 인터넷에서 관련 자료를 검색하고, 조사 결과를 문서화한다. 이 과정에서 AI 에이전트의 자율성, 환경 상호작용, LLM 활용...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  목표 최적화 완료
📝 [응답 최적화] 과거 회고를 고려한 응답 형식 정의 시작
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
  관련 과거 회고 2개 발견
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  응답 형식 정의 완료
✅ [1단계: 목표 설정] 완료

📋 [2단계: 목표 분해] 시작
📋 [목표 분해] 과거 회고를 고려한 태스크 분해 시작
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/

## 7. 역할 기반 협력

In [6]:
!python -m role_based_cooperation.main --task "AI agent 만들기 실습"

🎬 Role-Based Cooperation Agent 시작
사용자 쿼리: AI agent 만들기 실습

📋 [1단계: 계획 수립] 시작
📋 [계획 수립] 목표를 태스크로 분해 중...
📋 [단계 2] 목표 분해 시작
  목표: AI agent 만들기 실습...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
✅ 목표 분해 완료: 총 3개의 태스크 생성
  태스크 1: AI 에이전트의 기본 개념과 작동 방식을 이해하기 위해 인터넷에서 관련 자료를 조사한다....
  태스크 2: AI 에이전트를 만들기 위한 프로그래밍 언어와 도구를 선택하기 위해 인터넷에서 정보를 수집한다....
  태스크 3: 선택한 프로그래밍 언어와 도구를 사용하여 간단한 AI 에이전트를 개발하는 튜토리얼을 인터넷에서 찾아 따라한다....
  총 3개의 태스크 생성 완료

✅ [1단계: 계획 수립] 완료

👥 [2단계: 역할 배정] 시작
👥 [역할 배정] 각 태스크에 적합한 역할 배정 중...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  역할 배정 완료:
    태스크 1: AI 탐험가
    태스크 2: 기술 사냥꾼
    태스크 3: 튜토리얼 항해사

✅ [2단계: 역할 배정] 완료

⚙️  [3단계: 태스크 실행] 시작
📝 태스크 1/3 실행 중
⚙️  [태스크 실행] 역할: AI 탐험가
  태스크: AI 에이전트의 기본 개념과 작동 방식을 이해하기 위해 인터넷에서 관련 자료를 조사한다....
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  ✓ 실행 완료 